In [10]:
import re
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [3]:

def load_csv_data(folder_path):

    #finds teh csv files
    csv_files = [file for file in os.listdir(folder_path) if file.endswith(".csv")]


    csv_files = [os.path.join(folder_path, file) for file in csv_files]

# print the found files
    print(csv_files)

    dfs_csv = [pd.read_csv(file, sep=None, encoding="utf-16", skiprows =60) for file in csv_files]

    return dfs_csv

In [58]:

df1= pd.read_excel('Østbirk/Raw data/2022_08_30 to 09_20.xlsx')
df1= df1.drop(columns=['Scan'])
        
df2= pd.read_excel('Østbirk/Raw data/2022_11_03 to 2023_02_07.xlsx')
df3=pd.read_excel('Østbirk/Raw data/2023_02_09 to 06_16.xlsx')

df3= df3.drop(columns=['Scan'])

In [ ]:
print(df1.shape)
print(df2.shape)
print(df3.shape)
all_ex_data = pd.concat([df1,df2,df3],ignore_index=True,join='outer')
print(all_ex_data.shape)


In [ ]:
all_ex_data['Time'] = pd.to_datetime(all_ex_data['Time'], format="%d-%m-%Y %H:%M:%S:%f").dt.strftime("%Y-%m-%d %H:%M")
print(all_ex_data.head(5))

In [ ]:
dfs_csv = load_csv_data('Østbirk/Raw data')
print(len(dfs_csv))
for df in dfs_csv:
    print(f"DataFrame shape {df.shape}")


In [ ]:
all_csv_data = pd.concat(
    [df.drop(columns=['Scan'], errors='ignore') for df in dfs_csv],
    ignore_index=True,
    join='outer')

print(all_csv_data.shape)
print(all_csv_data.columns)

In [ ]:
all_csv_data['Time'] = pd.to_datetime(all_csv_data['Time'], format='%d-%m-%Y %H:%M:%S').dt.strftime("%Y-%m-%d %H:%M")
print(all_csv_data['Time'].head(5))

In [ ]:
raw_data_Østbirk = pd.concat([all_csv_data, all_ex_data], ignore_index=True, join='outer')
print(raw_data_Østbirk.shape)

In [ ]:
raw_data_Østbirk.columns = [col if col == "Time" else col.split()[0] for col in raw_data_Østbirk.columns]

raw_data_Østbirk = raw_data_Østbirk.sort_values(by='Time', ascending=True)
raw_data_Østbirk = raw_data_Østbirk.reset_index(drop=True)
print(raw_data_Østbirk.head(10))

In [ ]:
raw_data_Østbirk.dtypes

In [67]:
for col in raw_data_Østbirk.columns:
    if col != "Time":  
        raw_data_Østbirk[col] = (
            raw_data_Østbirk[col]
            .astype(str)  
            .str.replace(",", ".", regex=True) 
            .str.strip()
            .replace(["None", ""], np.nan) 
            .astype(float)  # convert to float
        )

In [ ]:
print(raw_data_Østbirk.dtypes)
print(raw_data_Østbirk.info())

In [70]:
raw_data_Østbirk.to_csv('all_rawData_Østbirk_sortedTime.csv', index=False)

Weather data

In [ ]:
df4= pd.read_csv('Østbirk/Weather data/2023_05_15 to 2024_05_15.csv', encoding='ISO-8859-1', sep=",", skiprows =5)
df5 =pd.read_csv('Østbirk/Weather data/2024_05_01 to 2025_01_30.csv', encoding='ISO-8859-1', sep=",", skiprows =5)

print(df4.shape)
print(df5.shape)

In [ ]:
all_weather_data = pd.concat([df4,df5,],ignore_index=True,join='outer')
print(all_weather_data.shape)
print(all_weather_data.head())

In [ ]:

all_weather_data['Date & Time'] = pd.to_datetime(all_weather_data['Date & Time'], format="%d/%m/%y %H:%M").dt.strftime("%Y-%m-%d %H:%M")

# Sort DataFrame by 'time' 
all_weather_data= all_weather_data.sort_values(by='Date & Time', ascending=True)

all_weather_data = all_weather_data.reset_index(drop=True)
all_weather_data.rename(columns={"Date & Time": "Time"}, inplace=True)


print(all_weather_data['Time'].head())
print(all_weather_data.shape)
print(all_weather_data.dtypes)

In [ ]:
all_weather_data.replace("--", np.nan, inplace=True)
for col in all_weather_data.columns:
    if col != 'Time' and col != 'High Wind Direction' and col != 'Prevailing Wind Direction':  #r
        all_weather_data[col] = (
            all_weather_data[col]
            .astype(str) 
            .str.replace(",", ".", regex=True) 
            .str.strip()  
            .astype(float) 
        )

In [ ]:
all_weather_data.to_csv('Østbirk_Allweather_data_sortedTime.csv', index=False)

Merge the datasets

In [ ]:
raw_data_Østbirk= pd.read_csv('all_rawData_Østbirk_sortedTime.csv')
raw_data_Østbirk['Time'] = pd.to_datetime(raw_data_Østbirk['Time'], errors='coerce')
(raw_data_Østbirk.dtypes)

In [ ]:
first_time = raw_data_Østbirk["Time"].iloc[0]
last_time = raw_data_Østbirk["Time"].iloc[-1]
print("Firt timestamp:", first_time)
print("Last timestamp:", last_time)

In [14]:
all_weather_data= pd.read_csv('Østbirk_Allweather_data_sortedTime.csv')

In [ ]:
first_time = all_weather_data["Time"].iloc[0]
last_time = all_weather_data["Time"].iloc[-1]
print("First timestamp:", first_time)
print("Last timestamp:", last_time)

In [ ]:
raw_data_Østbirk= raw_data_Østbirk.set_index("Time")
raw_data_filtered = raw_data_Østbirk.loc["2023-05-15 11:30":]
print(raw_data_filtered.index.min())  
print(raw_data_filtered.index.max())  

In [ ]:
for column in raw_data_filtered.columns:
    plt.figure(figsize=(50, 10))
    plt.plot(raw_data_filtered.index, raw_data_filtered[column], marker='o', linestyle='-')
    plt.xlabel("Tid")
    plt.ylabel(column)
    plt.title(f"Tidsserie af {column} ")
    plt.grid()
    plt.show()

In [20]:
start_date = "2023-05-15 11:30"
end_date = "2023-11-30"
raw_data_filtered= raw_data_filtered.loc[start_date:end_date]

In [ ]:
print(raw_data_filtered.index.min())  
print(raw_data_filtered.index.max())  

In [22]:
raw_data_filtered = raw_data_filtered.reset_index()

In [24]:
all_weather_data.set_index("Time", inplace=True)
start_date = "2023-05-15 11:30"
end_date = "2023-11-30 21:40"

Østbirk_weather_filtered = all_weather_data.loc[start_date:end_date].reset_index()

In [ ]:
#finds missing times
Østbirk_weather_filtered["Time"] = pd.to_datetime(Østbirk_weather_filtered["Time"])


expected_time_index = pd.date_range(start=start_date, end=end_date, freq="30min")


missing_times = expected_time_index.difference(Østbirk_weather_filtered["Time"])


print("Manglende tidspunkter:", missing_times)
print(f"Antal manglende tidspunkter: {len(missing_times)}")


In [26]:

Østbirk_weather_filtered= Østbirk_weather_filtered.set_index("Time") 
Østbirk_weather_filtered = Østbirk_weather_filtered.asfreq('30min')
Østbirk_weather_filtered = Østbirk_weather_filtered.reset_index()

In [27]:
Østbirk_weather_filtered= Østbirk_weather_filtered.iloc[1:]

In [ ]:
print(Østbirk_weather_filtered.isna().sum())

In [29]:
Østbirk_weather_filtered=Østbirk_weather_filtered[['Time','Temp - °C', 'High Temp - °C','Low Temp - °C','Hum - %','Dew Point - °C','Avg Wind Speed - m/s',
                                                   'Prevailing Wind Direction','Wind Run - m','High Wind Speed - m/s','High Wind Direction',
                                                   'Wind Chill - °C','Heat Index - °C','THW Index - °C','THSW Index - °C','Barometer - hPa',
                                                   'Rain - mm','High Rain Rate - mm/h','Solar Rad - W/m^2','High Solar Rad - W/m^2','UV Index',
                                                   'UV Dose - MEDs','High UV Index','Heating Degree Days','Cooling Degree Days','Inside Temp - °C',
                                                   'Inside Hum - %','Inside Dew Point - °C','Inside Heat Index - °C','ET - mm'
                                                   ]]

In [30]:
Østbirk_weather_filtered.columns= ['Time', 'Temp Out', 'Hi Temp', 'Low Temp', 'Out Hum',
       'Dew Pt.', 'Wind Speed', 'Wind Dir','Wind Run', 'Hi Speed', 'Hi Dir',
       'Wind Chill', 'Heat Index', 'THW Index','THSW Index', 'Bar', 'Rain',
       'Rain Rate', 'Solar Rad.', 'Hi Solar Rad.',
       'UV Index', 'UV Dose', 'Hi UV', 'Heat D-D','Cool D-D', 'In Temp', 'In Hum',
       'In Dew', 'In Heat', 'ET']

In [31]:
#fills the the NaN values in the wind direction columns with 'Calm' if wind speed is 0.0
Østbirk_weather_filtered.loc[(Østbirk_weather_filtered["Wind Dir"].isna()) & (Østbirk_weather_filtered["Wind Speed"] == 0.0), "Wind Dir"] = "Calm"
Østbirk_weather_filtered.loc[(Østbirk_weather_filtered["Hi Dir"].isna()) & (Østbirk_weather_filtered["Hi Speed"] == 0.0), "Hi Dir"] = "Calm"

In [32]:
Østbirk_weather_cleaned = Østbirk_weather_filtered.dropna(subset=['Temp Out'])
print(Østbirk_weather_cleaned.shape)

In [ ]:
print(Østbirk_weather_cleaned.isna().sum())

In [ ]:
for column in raw_data_filtered.columns:
    plt.figure(figsize=(50, 10))
    plt.plot(raw_data_filtered.index, raw_data_filtered[column], marker='o', linestyle='-')
    plt.xlabel("Tid")
    plt.ylabel(column)
    plt.title(f"Tidsserie af {column} ")
    plt.grid()
    plt.show()

In [ ]:
#selects 10 sensors
Østbirk_raw_data_filtered2= raw_data_filtered[['Time','103','102','117','217','105','111','120','205','210','215']]
Østbirk_raw_data_filtered2.rename(columns={'102': 'Gip Top Center','103': 'Gip Bottom Center', '117':'Clading aPV east','217': 'Sash 15 mm below pane',
                                          '105': 'Inner pane spacer - top center','111': 'Gip aPV east corner','120': 'Outer frame upper - top center',
                                          '205':'Inner frame - top center','210': 'Top frame uPV east', '215':'Sash front bottom center'
                                          }, inplace=True)


In [ ]:
#selects 10 random sensors to the robustness analysis
Østbirk_raw_data_filtered3= raw_data_filtered[['Time','115','219','118','106','112','201','211','207','109','305']]
Østbirk_raw_data_filtered3.rename(columns={ '115':'Cladding uPV east corner','219': 'Sash 15 mm below pane - bottom center', '118': 'Gip uPV east',
                                          '106':'Outer pane spacer - top center','112': 'Gip uPV west corner','201': 'Outer frame lower - top center', 
                                          '211':'Sash behind pane - aPV east', '207': 'Inner frame uPV east', '109':'Inner pane spacer - bottom center', '305': 'Sash gasket level - bottom center',
                                          }, inplace=True)

In [ ]:
Østbirk_raw_data_filtered.isna().sum()

In [50]:
#merges the sensor data and weather data with a 10 min tollerance
def merge_dfs(df,df2):
    data = pd.merge_asof(df,  
                          df2,   
                          left_index=True, 
                          right_index=True,
                          direction="nearest",  
                          tolerance=pd.Timedelta("10min"))  
    return data

In [44]:
Østbirk_weather_cleaned=Østbirk_weather_cleaned.set_index('Time')

In [84]:
Østbirk_raw_data_filtered2=Østbirk_raw_data_filtered2.set_index('Time')
Østbirk_df2=merge_dfs(Østbirk_weather_cleaned,Østbirk_raw_data_filtered2)

In [52]:
Østbirk_raw_data_filtered3=Østbirk_raw_data_filtered3.set_index('Time')
Østbirk_df3=merge_dfs(Østbirk_weather_cleaned,Østbirk_raw_data_filtered3)

In [86]:
Østbirk_data2=Østbirk_df2.dropna(subset='Gip Bottom Center')

In [54]:
Østbirk_data3=Østbirk_df3.dropna(subset='Gip uPV east')

In [90]:
def missing_times(df, start_date, end_date):
    expected_time_index = pd.date_range(start=start_date, end=end_date, freq="30min")
    missing_times = expected_time_index.difference(df['Time'])
    print("Missing:", missing_times)
    print(f"Number of missing times: {len(missing_times)}")
    return missing_times



In [ ]:

def plot_missing_times(m):
    plt.figure(figsize=(12, 6))
    plt.scatter(m, [1] * len(m), color='blue', marker='|', s=10)
    plt.xlabel("Time")
    plt.title("Missing timestamps")
    plt.yticks([])
    plt.xticks(rotation=45)
    plt.show()

Østbirk_data=Østbirk_data.reset_index('Time')
m=missing_times(Østbirk_data, '2023-05-15 12:00:00','2023-11-30 21:30:00')
plot_missing_times(m)

In [ ]:
Østbirk_data2=Østbirk_data2.reset_index()

In [ ]:

Østbirk_data2["Window_Type"] = 1
Østbirk_data2["Blinds"] = 1
Østbirk_data2["Window_ID"] = 1
Østbirk_data2['Latitude']=
Østbirk_data2['Longitude']= 

In [95]:
Østbirk_data2.to_csv("df_Østbirk_10.csv",index=False)

In [ ]:
Østbirk_data3["Window_Type"] = 1
Østbirk_data3["Blinds"] = 1
Østbirk_data3["Window_ID"] = 1
Østbirk_data3['Latitude']= 
Østbirk_data3['Longitude']=

In [57]:
Østbirk_data3=Østbirk_data3.reset_index('Time')

In [58]:
Østbirk_data3.to_csv("df_Østbirk_random.csv",index=False)